## Imports

In [ ]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [ ]:
from pymongo import MongoClient
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import string

In [ ]:
from sshtunnel import SSHTunnelForwarder
from getpass import getpass

## Connnection

In [ ]:
MONGO_HOST = "flask01.network.ncf.edu"
MONGO_DB = "hfmil"
MONGO_USER = input("Enter your username: ")
MONGO_PASS = getpass("Enter your password: ")

In [ ]:
server = SSHTunnelForwarder(
    MONGO_HOST,
    ssh_username=MONGO_USER,
    ssh_password=MONGO_PASS,
    remote_bind_address=('127.0.0.1', 27017)
)

In [ ]:
server.start()

In [ ]:
client = MongoClient('127.0.0.1', server.local_bind_port)

In [ ]:
db = client[MONGO_DB]

In [ ]:
db.list_collection_names()

## Conflict Data

In [ ]:
cur = db.conflicts.find({},{'_id':0,'location':1,'side_a':1,'side_b':1,'year':1, 'start_date':1, 'ep_end_date':1, 'region':1})

In [ ]:
df = pd.DataFrame()
for i in range(cur.count()):
    temp = cur.next()
    df = df.append(temp, ignore_index=True)

In [ ]:
df

In [ ]:
# cleans the text to isolate country name
def cleanSide(side):
    nopunc = [c for c in side if c not in string.punctuation]
    nopunc = ''.join(nopunc)
    return [word for word in nopunc.split() if word.lower() not in ['government','of','the']]

In [ ]:
df['side_a'] = df['side_a'].apply(cleanSide)

In [ ]:
df['side_b'] = df['side_b'].apply(cleanSide)

In [ ]:
df['location'] = df['location'].apply(cleanSide)

In [ ]:
df.head()

## Connecting Conflicts and HFI

In [ ]:
cur = db.hfi19.find({},{'_id':0,'ISO_code':1,'countries':1,'hf_score':1, 'year':1})

In [ ]:
hfDf = pd.DataFrame()
for i in range(cur.count()):
    temp = cur.next()
    hfDf = hfDf.append(temp, ignore_index=True)

In [ ]:
hfDf = hfDf.rename({'countries':'country'}, axis = 1)

In [ ]:
hfDf.head()

## Query conflicts side A

In [ ]:
def getConflicts(country):
    cur = db.conflicts.find({'side_a': {'$regex': country, '$options': 'i'}},{'_id':0, 'conflict_id':1})
    conflicts = set()
    while True:
        try:
            temp = cur.next()
            if len(temp) > 0:
                conflicts.add(temp['conflict_id'])
        except:
            break
    return list(conflicts)

In [ ]:
def getNumConflicts(country):
    cur = db.conflicts.find({'side_a': {'$regex': country, '$options': 'i'}},{'_id':0, 'conflict_id':1})
    conflicts = set()
    while True:
        try:
            temp = cur.next()
            if len(temp) > 0:
                conflicts.add(temp['conflict_id'])
        except:
            break
    return len(list(conflicts))

In [ ]:
def getNumOp(country):
    cur = db.conflicts.find({'side_a': {'$regex': country, '$options': 'i'}},{'_id':0, 'conflict_id':1})
    conflicts = []
    while True:
        try:
            temp = cur.next()
            if len(temp) > 0:
                conflicts.append(temp['conflict_id'])
        except:
            break
    return len(conflicts)

In [ ]:
hfDf['num_conflicts_side_A'] = hfDf['country'].apply(getNumConflicts)

In [ ]:
hfDf['conflicts_side_A'] = hfDf['country'].apply(getConflicts)

In [ ]:
hfDf['num_ops_side_A'] = hfDf['country'].apply(getNumOp)

In [ ]:
hfDf.sort_values(by = "num_conflicts_side_A", ascending = False)

## Conflicts per country side A
(for countries with more than one conflict)

In [ ]:
plt.figure(figsize=(15,6))
sns.barplot(data = hfDf[hfDf['num_conflicts_side_A'] > 0].sort_values('num_conflicts_side_A',ascending=False), 
            x = 'ISO_code', y = 'num_conflicts_side_A',
           color = '#3eb077')
plt.xticks(rotation = 90)
plt.tick_params(axis='x', which='major', labelsize=10)
plt.tight_layout(h_pad=2)
plt.savefig('num_conflicts_side_A.png')

## Operation per country side A
(most conflicts are composed of more than one operation)

In [ ]:
plt.figure(figsize=(15,6))
sns.barplot(data = hfDf[hfDf['num_ops_side_A'] > 0].sort_values('num_ops_side_A',ascending=False), 
            x = 'ISO_code', y = 'num_ops_side_A',
           color = '#3eb077')
plt.xticks(rotation = 90)
plt.tick_params(axis='x', which='major', labelsize=10)
plt.tight_layout(h_pad=2)
plt.savefig('num_ops_side_A.png')

## HF score vs. number of conflicts side A

In [ ]:
d17 = hfDf[hfDf.year == 2017]
plt.figure(figsize=(10,5))
sns.scatterplot(data=d17, x = 'num_conflicts_side_A', y = 'hf_score', size='num_ops_side_A')

## Operations per country choropleth
(where country is side A)

In [ ]:
p = px.choropleth(data_frame=hfDf, locations='ISO_code', color= 'num_ops_side_A')
p.show()

## Query conflicts side B

In [ ]:
def getConflicts(country):
    cur = db.conflicts.find({'side_b': {'$regex': country, '$options': 'i'}},{'_id':0, 'conflict_id':1})
    conflicts = set()
    while True:
        try:
            temp = cur.next()
            if len(temp) > 0:
                conflicts.add(temp['conflict_id'])
        except:
            break
    return list(conflicts)

In [ ]:
def getNumConflicts(country):
    cur = db.conflicts.find({'side_b': {'$regex': country, '$options': 'i'}},{'_id':0, 'conflict_id':1})
    conflicts = set()
    while True:
        try:
            temp = cur.next()
            if len(temp) > 0:
                conflicts.add(temp['conflict_id'])
        except:
            break
    return len(list(conflicts))

In [ ]:
def getNumOp(country):
    cur = db.conflicts.find({'side_b': {'$regex': country, '$options': 'i'}},{'_id':0, 'conflict_id':1})
    conflicts = []
    while True:
        try:
            temp = cur.next()
            if len(temp) > 0:
                conflicts.append(temp['conflict_id'])
        except:
            break
    return len(conflicts)

In [ ]:
hfDf['num_conflicts_side_B'] = hfDf['country'].apply(getNumConflicts)

In [ ]:
hfDf['conflicts_side_B'] = hfDf['country'].apply(getConflicts)

In [ ]:
hfDf['num_ops_side_B'] = hfDf['country'].apply(getNumOp)

In [ ]:
hfDf.sort_values(by = "num_conflicts_side_B", ascending = False)

## Conflicts per country side B
(for countries with more than one conflict)

In [ ]:
plt.figure(figsize=(15,6))
sns.barplot(data = hfDf[hfDf['num_conflicts_side_B'] > 0].sort_values('num_conflicts_side_B',ascending=False), 
            x = 'ISO_code', y = 'num_conflicts_side_B',
           color = '#3eb077')
plt.xticks(rotation = 90)
plt.tick_params(axis='x', which='major', labelsize=10)
plt.tight_layout(h_pad=2)
plt.savefig('num_conflicts_side_B.png')

## Operation per country side B
(most conflicts are composed of more than one operation)

In [ ]:
plt.figure(figsize=(15,6))
sns.barplot(data = hfDf[hfDf['num_ops_side_B'] > 0].sort_values('num_ops_side_B',ascending=False), 
            x = 'ISO_code', y = 'num_ops_side_B',
           color = '#3eb077')
plt.xticks(rotation = 90)
plt.tick_params(axis='x', which='major', labelsize=10)
plt.tight_layout(h_pad=2)
plt.savefig('num_ops_side_B.png')

## Adding military expenditure proportion

In [ ]:
years = list(hfDf.year.unique())
years

In [ ]:
tempdf = pd.DataFrame()
for year in years:
    codes = hfDf.loc[hfDf['year']==year,'ISO_code']
    for code in codes:
        prop = db.exp_prop.find_one({'Code':code},{'_id':0, str(int(year)):1})
        try:
            prop = list(prop.values())[0]
        except:
            prop = None
        tempdf = tempdf.append({'ISO_code':code, 'year':year,'prop_gdp':prop}, 
                               ignore_index=True)

In [ ]:
tempdf

In [ ]:
hfDf = hfDf.merge(tempdf, on = ['ISO_code', 'year'])

In [ ]:
hfDf

In [ ]:
hfDf.describe()

## HF score per year

In [ ]:
fig = go.Figure()

years = [year for year in hfDf.year.unique()]

for year in years:
    fig.add_trace(
        go.Choropleth(
            z = hfDf.loc[hfDf['year']==year,'hf_score'],
            locations = hfDf.loc[hfDf['year']==year,'ISO_code'],
            zmax = 9.12,
            zmin = 3.5))

fig.data[9].visible = True


steps = []
for i in range(len(years)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "HF scores for the year: " + str(int(years[i]))}],
        label = str(int(years[i]))
    )
    step["args"][0]["visible"][i] = True
    steps.append(step)
    
sliders = [dict(
    active=10,
    currentvalue={"prefix": "Year: "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders
)

fig.show()

 ## Military Expenditure Per Year

In [ ]:
fig = go.Figure()

years = [year for year in hfDf.year.unique()]

for year in years:
    fig.add_trace(
        go.Choropleth(
            z = hfDf.loc[hfDf['year']==year,'prop_gdp'],
            locations = hfDf.loc[hfDf['year']==year,'ISO_code'],
            zmax = 0.1,
            zmin = 0))

fig.data[9].visible = True


steps = []
for i in range(len(years)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Military expenditure as proportion of gdp for the year: " + str(int(years[i]))}],
        label = str(int(years[i]))
    )
    step["args"][0]["visible"][i] = True
    steps.append(step)
    
sliders = [dict(
    active=10,
    currentvalue={"prefix": "Year: "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders
)

fig.show()

## Conflict info

In [ ]:
ic_df = hfDf.loc[hfDf.year == 2017,['ISO_code', 'conflicts_side_A', 'conflicts_side_B']]

In [ ]:
con_info = pd.DataFrame()
for i in range(len(ic_df)):
    temp = ic_df.iloc[i,:]
    conflicts = temp['conflicts_side_A']
    conflicts.append(temp['conflicts_side_B'])
    code = temp['ISO_code']
    for con in conflicts:
        cur = db.conflicts.find({'conflict_id':con}, 
                               {'_id':0, 'location':1,'side_a':1,
                                'side_b':1, 'year':1,'intensity_level':1,
                                'cumulative_intensity':1,
                                'start_date':1,'ep_end_date':1
                               })
        while True:
            try:
                nxt = cur.next()
                nxt['ISO_code'] = code
                nxt['conflict_id'] = con
                con_info = con_info.append(nxt, ignore_index = True)
            except:
                break

In [ ]:
con_info.describe()

In [ ]:
con_info.head()

In [ ]:
con_info.describe()

In [ ]:
con_info.sort_values('year', ascending=False, inplace=True)

In [ ]:
con_info.sort_values('year', ascending=False, inplace=True)

fig = go.Figure()

years = [year for year in con_info.year.unique()]


for year in years:
    fig.add_trace(
        go.Choropleth(
            z = con_info.loc[con_info['year']==year,'cumulative_intensity'],
            locations = hfDf.loc[con_info['year']==year,'ISO_code'],
            zmax = 1,
            zmin = 0,
            text = con_info.loc[:,['conflict_id','location','side_a','side_b']]
        ))

fig.data[-1].visible = True


steps = []
for i in range(len(years)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Cumulative conflict intensity level per year: " + str(int(years[i]))}],
        label = str(int(years[i]))
    )
    step["args"][0]["visible"][i] = True
    steps.append(step)
    
sliders = [dict(
    active=10,
    currentvalue={"prefix": "Year: "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders
)

fig.show()

In [ ]:
con_info.sort_values('year', ascending=False, inplace=True)

fig = go.Figure()

years = [year for year in con_info.year.unique()]


for year in years:
    fig.add_trace(
        go.Choropleth(
            z = con_info.loc[con_info['year']==year,'intensity_level'],
            locations = con_info.loc[con_info['year']==year,'ISO_code'],
            zmax = 2,
            zmin = 1,
            text = con_info.loc[:,['conflict_id','location','side_a','side_b']]
            ))

fig.data[-1].visible = True


steps = []
for i in range(len(years)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Conflict intensity level per year: " + str(int(years[i]))}],
        label = str(int(years[i]))
    )
    step["args"][0]["visible"][i] = True
    steps.append(step)
    
sliders = [dict(
    active=10,
    currentvalue={"prefix": "Year: "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders
)

fig.show()

In [ ]:
client.close()

In [ ]:
server.close()

In [ ]:
server.is_active